In [1]:
from pyspark.sql import SparkSession


In [3]:
spark = SparkSession.builder.appName("rdd").getOrCreate()

# lower level api 
- rdd 
- accumulatorw
- broadcast

# rdd 


In [4]:
collection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")

words = spark.sparkContext.parallelize(collection,2)

In [5]:
words.getNumPartitions()

2

In [6]:
words.collect()

['Spark',
 'The',
 'Definitive',
 'Guide',
 ':',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

In [7]:
words.setName("mywords")

mywords ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262

In [8]:
words.name

<bound method RDD.name of mywords ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262>

In [9]:
words.take(2)

['Spark', 'The']

## transoformation
1. distinct 
2. map 
3. filter 
4. flatmap 
5. sort 
6. random split 

## action 
1. reduce 
2. count / countApprox
3. countByValue
4. first
5. max and min 
6. take
7.  

## saving files 
1. saveAsTextFile
2. sequence file saveAsObjectFile
3. 

## caching
wrods.cache()

# checkpointing

##  partition
1. mapPartition
2. foreachPartition
3. glom 

In [10]:
words.glom().collect()

[['Spark', 'The', 'Definitive', 'Guide', ':'],
 ['Big', 'Data', 'Processing', 'Made', 'Simple']]